Cette Partie est dédiée à la Transformation de notre Texte et donc à la création de features exploitable pour le Machine Learning.
Nous allons Testser différentes méthodes de représentation de nos document :
(1) Méthodes de comptage direct :
    - Approche Bag of Words
    - Approche n-gram (Bag of words généralisé)
    - Approche TF-IDF

(2) Méthode plongement de mot

# <font color = Green>Partie 1 : n-gram et TF-IDF</font>

## <font color = purple>1. Préléminaires</font>

### <font color = grey>1.1 Contexte</font>

TEXTE INTRO

### <font color = grey>1.2 Librairies</font>

In [5]:
import nltk
import re
import numpy as np

import numpy as np
import pandas as pd

### <font color = grey>1.3 Chargement des données</font>

In [6]:
#Chemin de notre fichier source
data = "cleaned/"
os.listdir(data)

#Chargement du Dataframe
fn = data + 'final_df.csv'
df = pd.read_csv(fn)
df.head()

,Title,Body,Tags,clean_title
0,"Differences between declare, typeset and local...","<p>When typing variables in Bash, what is the ...",<bash><shell><variables><scope><declare>,differ declar local variabl bash
1,Dealing with PHP server and MySQL server in di...,<p>For those of us who use standard shared hos...,<php><mysql><datetime><date><timezone>,deal php server mysql server differ time
2,Difference between async/await and ES6 yield w...,<p>I was just reading this fantastic article «...,<javascript><node.js><ecmascript-6><generator>...,differ async await yield gener
3,Loading System.ServiceModel configuration sect...,"<p>Using C# .NET 3.5 and WCF, I'm trying to wr...",<c#><.net><xml><wcf><configurationmanager>,load system configur section use
4,XML Serialization and Inherited Types,"<p>Following on from my <a href=""https://stack...",<c#><xml><inheritance><serialization><xml-seri...,xml serial inherit type


-----

## <font color = purple>2. approche Bag of Words</font>

A AJOUTER DANS LE PREMIER NOTEBOOK

In [21]:
is_NaN = df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df[row_has_NaN]
rows_with_NaN

,Title,Body,Tags,clean_title


In [20]:
df = df.dropna()

In [10]:
CountVec = CountVectorizer(ngram_range=(1,1)) # to use bigrams ngram_range=(2,2)

In [22]:
#transform
Count_data = CountVec.fit_transform(df["clean_title"])


In [23]:
#create dataframe
cv_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())

       aac  aadhaar  aapt  aar  ab  abaddressbook  abbrevi  abi  abl  abort  \
0        0        0     0    0   0              0        0    0    0      0   
1        0        0     0    0   0              0        0    0    0      0   
2        0        0     0    0   0              0        0    0    0      0   
3        0        0     0    0   0              0        0    0    0      0   
4        0        0     0    0   0              0        0    0    0      0   
...    ...      ...   ...  ...  ..            ...      ...  ...  ...    ...   
27612    0        0     0    0   0              0        0    0    0      0   
27613    0        0     0    0   0              0        0    0    0      0   
27614    0        0     0    0   0              0        0    0    0      0   
27615    0        0     0    0   0              0        0    0    0      0   
27616    0        0     0    0   0              0        0    0    0      0   

       ...  zone  zookeep  zoom  zoomabl  zoomtable

c:\Users\Anthony\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [25]:
#Il s'agit de X 
cv_dataframe.head(10)

,aac,aadhaar,aapt,aar,ab,abaddressbook,abbrevi,abi,abl,abort,...,zone,zookeep,zoom,zoomabl,zoomtablemanag,zpl,zsh,zuul,zxing,zzabn
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents
text = ["The quick brown fox jumped over the lazy dog."]
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

## <font color = purple>3. approche n-gram </font>

stop_words: Since CountVectorizer just counts the occurrences of each word in its vocabulary, extremely common words like ‘the’, ‘and’, etc. will become very important features while they add little meaning to the text. Your model can often be improved if you don’t take those words into account. Stop words are just a list of words you don’t want to use as features. You can set the parameter stop_words=’english’ to use a built-in list. Alternatively you can set stop_words equal to some custom list. This parameter defaults to None.
ngram_range: An n-gram is just a string of n words in a row. E.g. the sentence ‘I am Groot’ contains the 2-grams ‘I am’ and ‘am Groot’. The sentence is itself a 3-gram. Set the parameter ngram_range=(a,b) where a is the minimum and b is the maximum size of ngrams you want to include in your features. The default ngram_range is (1,1).
In a recent project where I modeled job postings online, I found that including 2-grams as features boosted my model’s predictive power significantly. This makes intuitive sense; many job titles such as ‘data scientist’, ‘data engineer’, and ‘data analyst’ are 2 words long.
min_df, max_df: These are the minimum and maximum document frequencies words/n-grams must have to be used as features. If either of these parameters are set to integers, they will be used as bounds on the number of documents each feature must be in to be considered as a feature. If either is set to a float, that number will be interpreted as a frequency rather than a numerical limit. min_df defaults to 1 (int) and max_df defaults to 1.0 (float).
max_features: This parameter is pretty self-explanatory. The CountVectorizer will choose the words/features that occur most frequently to be in its’ vocabulary and drop everything else.
You would set these parameters when initializing your CountVectorizer object as shown below.

# <font color = Green>Partie 2 : Plongement de Mot</font>